In [12]:
import os
import pandas as pd 
import numpy as np

import tensorflow 
from tensorflow import keras
from keras import layers, regularizers

import cv2 #images
import psycopg2


from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## import dependencies for model
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical, normalize
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

from sklearn.metrics import classification_report, confusion_matrix

In [13]:
# Database connection setup

import json

with open('config.json', 'r') as config_file:
    config = json.load(config_file)

DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]
DB_USER = config["DB_USER"]
DB_PASS = config["DB_PASS"]


In [14]:
# import pandas as pd
# import psycopg2
# from PIL import Image
# import io
# import base64

# # Connect to the database
# connection = psycopg2.connect(
#             host=DB_HOST,
#             database=DB_NAME,
#             user=DB_USER,
#             password=DB_PASS
# )

# cursor = connection.cursor()

# # Read the metadata CSV
# metadata = pd.read_csv('/Users/Jack/Desktop/Skin-Cancer-Detection-Project-4/Dataset/HAM10000_metadata.csv')

# # Function to convert image to binary data
# def image_to_binary(image_path):
#     with open(image_path, 'rb') as file:
#         binary_data = file.read()
#     return binary_data

# # Base paths for the image directories
# image_dir_1 = '/Users/Jack/Desktop/Skin-Cancer-Detection-Project-4/Dataset/HAM10000_images_part_1'
# image_dir_2 = '/Users/Jack/Desktop/Skin-Cancer-Detection-Project-4/Dataset/HAM10000_images_part_2'

# # Insert the data into the table
# for index, row in metadata.iterrows():
#     image_id = row['image_id']
    
#     # Check which directory contains the image
#     image_path_1 = os.path.join(image_dir_1, f"{image_id}.jpg")
#     image_path_2 = os.path.join(image_dir_2, f"{image_id}.jpg")
    
#     if os.path.exists(image_path_1):
#         image_path = image_path_1
#     elif os.path.exists(image_path_2):
#         image_path = image_path_2
#     else:
#         print(f"Image {image_id} not found in either directory.")
#         continue

#     try:
#         binary_data = image_to_binary(image_path)
#         cursor.execute(
#             """
#             INSERT INTO skin_cancer_data (lesion_id, image_id, dx, dx_type, age, sex, localization, image_data)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
#             """,
#             (row['lesion_id'], row['image_id'], row['dx'], row['dx_type'],
#              row['age'], row['sex'], row['localization'], psycopg2.Binary(binary_data))
#         )
#     except Exception as e:
#         print(f"Error processing image {image_id}: {e}")

# # Commit the transaction and close the connection
# connection.commit()
# cursor.close()
# connection.close()


In [15]:
import pandas as pd
import json
from sqlalchemy import create_engine

# Load database connection parameters from config.json
def load_config():
    with open('config.json', 'r') as file:
        config = json.load(file)
    return config

# Create a SQLAlchemy engine using parameters from config
def create_engine_connection():
    config = load_config()
    DB_USER = config["DB_USER"]
    DB_PASS = config["DB_PASS"]
    DB_HOST = config["DB_HOST"]
    DB_NAME = config["DB_NAME"]
    
    engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    return engine

# Function to load data from the database using SQLAlchemy engine
def load_data_from_db():
    engine = create_engine_connection()
    query = "SELECT * FROM skin_cancer_data;"
    df = pd.read_sql(query, engine)
    engine.dispose()  # Properly dispose of the engine when done
    return df

# Load the data and display the head of the DataFrame
df = load_data_from_db()
print(df.head())


     lesion_id      image_id   dx dx_type   age   sex localization  \
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp   
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp   
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp   
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp   
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear   

                                          image_data  
0  [b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ...  
1  [b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ...  
2  [b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ...  
3  [b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ...  
4  [b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ...  


In [16]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_data
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,"[b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,"[b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,"[b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,"[b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,"[b'\xff', b'\xd8', b'\xff', b'\xdb', b'\x00', ..."


In [17]:
df.to_csv('metadata_df.csv', index=False)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
 7   image_data    10015 non-null  object 
dtypes: float64(1), object(7)
memory usage: 626.1+ KB


In [19]:
df['dx'].unique()

array(['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec'], dtype=object)

In [20]:
from PIL import Image
import io
import numpy as np

def process_images_in_batches(df, batch_size=1015):
    total_images = len(df)
    batch = df.iloc[:batch_size]
   
    new_df = pd.DataFrame()  
    new_df['image_id'] = batch['image_id']
    new_df['image_array'] = None
    
    for idx, row in batch.iterrows():
        binary_data = row['image_data']
        with Image.open(io.BytesIO(binary_data)) as im: 
            image = im.resize((224, 224))
            image_array = np.array(image) / 255.0
            new_df['image_array'].iloc[idx]= image_array
            
    return new_df


In [21]:
# Call the function with your DataFrame
X_images = process_images_in_batches(df)
X_images.head()

/var/folders/w1/kv9m74sx7rn3jh6fkm7wrzrc0000gp/T/ipykernel_78558/509694628.py:18: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  new_df['image_array'].iloc[idx]= image_array
/var/folders/w1/kv9m74sx7rn3jh6fkm7wrzrc0000gp/T/ipykernel_78558/509

,image_id,image_array
0,ISIC_0027419,"[[[0.7333333333333333, 0.5882352941176471, 0.7..."
1,ISIC_0025030,"[[[0.09411764705882353, 0.050980392156862744, ..."
2,ISIC_0026769,"[[[0.7294117647058823, 0.4980392156862745, 0.5..."
3,ISIC_0025661,"[[[0.09019607843137255, 0.043137254901960784, ..."
4,ISIC_0031633,"[[[0.49019607843137253, 0.3254901960784314, 0...."


In [22]:
# Save the DataFrame as a CSV file
X_images.to_csv('skin_cancer_detection_array_all_images.csv', index=False)

In [23]:
X_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_id     1015 non-null   object
 1   image_array  1015 non-null   object
dtypes: object(2)
memory usage: 16.0+ KB


In [24]:
# Merge dataframes 
merged_df = pd.merge(X_images, df, on='image_id')


In [ ]:
merged_df.to_csv('merged_df.csv', index=False)

In [25]:
from sklearn.model_selection import train_test_split

# Define X (features) and y (target variable)
X = np.stack(merged_df['image_array'].values)  # Convert list of arrays to a 4D array
y = merged_df['dx'].values

In [26]:
# Encode labels 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [27]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [28]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(812, 224, 224, 3)
(812,)
(203, 224, 224, 3)
(203,)


In [29]:
# Create Image Data Generation

datagen = ImageDataGenerator(rescale=(1./255)
                             ,rotation_range=10
                             ,zoom_range = 0.1
                             ,width_shift_range=0.1
                             ,height_shift_range=0.1)

testgen = ImageDataGenerator(rescale=(1./255))

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Build CNN (conv neural network) Model
def build_cnn_model(input_shape, num_classes):
    model = Sequential()
    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten and Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Create the model
input_shape = (224, 224, 3)  # Example input shape, modify as needed
num_classes = len(np.unique(y_encoded))  # Set based on the number of classes in your dataset

cnn_model = build_cnn_model(input_shape, num_classes)

# Compile the model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# View the model summary
cnn_model.summary()


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 14, 14, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 128,964,930 (491.96 MB)

 Trainable params: 128,961,986 (491.95 MB)

 Non-trainable params: 2,944 (11.50 KB)

: 

In [31]:
#Training our CNN Model
cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=16)

Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 465s 9s/step - accuracy: 0.9125 - loss: 8.3546 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 444s 9s/step - accuracy: 0.9672 - loss: 9.2440 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/50
46/51 ━━━━━━━━━━━━━━━━━━━━ 41s 8s/step - accuracy: 0.9993 - loss: 107.7160

In [ ]:
train_score = model.evaluate(X_train, y_train, verbose= 1)
test_score = model.evaluate(X_test, y_test, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
# Make Predictions
predictions=model.predict_classes(y_train)

check=[]
for i in range(len(y_test)):
  for j in range(7):
    if(y_test[i][j]==1):
      check.append(j)
check=np.asarray(check)

print(classification_report(check, predictions))

In [ ]:
# from PIL import Image
# import io
# import numpy as np

# def process_images(df):
#     images = []
#     for binary_data in df['image_data']:
#         # Open the binary data as an image
#         image = Image.open(io.BytesIO(binary_data))
#         # Resize the image to 224x224
#         image = image.resize((224, 224))
#         # Convert the image to a numpy array and normalize pixel values
#         image_array = np.array(image) / 255.0
#         images.append(image_array)
#     return np.array(images)


In [ ]:
# from PIL import Image
# import io

# # Convert image binary data back into arrays and normalize pixel values
# def process_images(df):
#     images = []
#     for binary_data in df['image_data']:
#         image = Image.open(io.BytesIO(binary_data))
#         image = image.resize((224, 224))  # Resize to 224x224
#         image_array = np.array(image) / 255.0  # Normalize pixel values
#         images.append(image_array)
#     return np.array(images)

In [ ]:
# from PIL import Image
# import io
# import matplotlib.pyplot as plt

# # Function to display an image from binary data
# def display_image(image_binary):
#     image = Image.open(io.BytesIO(image_binary))
#     plt.imshow(image)
#     plt.axis('off')  # Hide axis
#     plt.show()

# # Retrieve a single image from the DataFrame
# sample_image_binary = df['image_data'].iloc[0]  # Get the first image for example

# # Display the image
# display_image(sample_image_binary)


In [ ]:
# # Process images and retrieve the image arrays
# X_images = process_images(df)

In [ ]:
# # Process the images and extract the labels
# X_images = process_images(df)
# y_labels = df['dx'].values

In [ ]:
# from sklearn.preprocessing import LabelEncoder, StandardScaler

# # Encode categorical variables
# df['sex'] = LabelEncoder().fit_transform(df['sex'])
# df['localization'] = LabelEncoder().fit_transform(df['localization'])
# df['dx'] = LabelEncoder().fit_transform(df['dx'])  # This will be the target variable

# # Normalize age
# scaler = StandardScaler()
# df['age'] = scaler.fit_transform(df[['age']])

# # Convert image binary data back into arrays and normalize pixel values
# def process_images(df):
#     images = []
#     for binary_data in df['image_data']:
#         image = Image.open(io.BytesIO(binary_data))
#         image = image.resize((224, 224))  # Resize to 224x224
#         image_array = np.array(image) / 255.0  # Normalize pixel values
#         images.append(image_array)
#     return np.array(images)

# X_images = process_images(df)
# y_labels = df['dx'].values
